<a href="https://colab.research.google.com/github/uol-mediaprocessing/group-projects-image-restauration/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Restoration Damaged Images

## Gruppenmitglieder:

- Sören Gade
- Florian Meyer



## Ziele

Folgende Zielsetzung soll bis zum Ende der Modullaufzeit erreicht werden:

1. Ziel 1
2. Ziel 2
3. Ziel 3